# Cleaning

### Motivacion
Dejar los datos en un archivo parquet para poder ser utilizados en el modelo de clasificacion de texto, para eso se eliminan las columnas que no son relevantes y se limpian los textos para que sean mas faciles de procesar.

### Importar librerias

In [58]:
import pandas as pd
from bs4 import BeautifulSoup
import re
import nltk

### Definicion de funciones

In [59]:
def read_data(file_path):
    """
    Lee un archivo CSV y lo carga en un DataFrame de pandas.

    Parámetros
    ----------
    file_path: str
        Ruta al archivo CSV que se desea leer.

    Retorna
    -------
    pandas.DataFrame
        DataFrame con los datos cargados desde el archivo CSV.
    """
    data = pd.read_csv(file_path, encoding="ISO-8859-1")
    return data

In [60]:
def drop_columns(df, columns):
    """
    Elimina una o varias columnas de un DataFrame de pandas.

    Parámetros
    ----------
    df: pandas.DataFrame
        DataFrame del cual se eliminarán las columnas.
    columns: str o list de str
        Nombre o lista de nombres de las columnas a eliminar del DataFrame.

    Retorna
    -------
    pandas.DataFrame
        El mismo DataFrame después de eliminar las columnas indicadas.
    """
    df.drop(columns=columns, inplace=True)
    return df

In [61]:
def merge_datasets(df_left, df_right, merge_column):
    """
    Combina dos DataFrames de pandas mediante un join interno sobre una columna común.

    Parámetros
    ----------
    df_left: pandas.DataFrame
        DataFrame izquierdo en la operación de merge.
    df_right: pandas.DataFrame
        DataFrame derecho en la operación de merge.
    merge_column: str
        Nombre de la columna sobre la cual se realizará la combinación. Debe existir en ambos DataFrames.

    Retorna
    -------
    pandas.DataFrame
        DataFrame resultante de la combinación de df_left y df_right.
    """
    df_merge = df_left.merge(df_right, on=merge_column)
    return df_merge

In [62]:
def group_tags(df, group_column, agg_column):
    """
    Agrupa un DataFrame de pandas por una columna y agrega los valores de otra columna en listas.

    Parámetros
    ----------
    df: pandas.DataFrame
        DataFrame a agrupar.
    group_column: str
        Nombre de la columna por la cual agrupar.
    agg_column: str
        Nombre de la columna cuyos valores se agregarán en listas por cada grupo.

    Retorna
    -------
    pandas.DataFrame
        DataFrame con las columnas `group_column` y `agg_column`, donde esta última contiene listas de valores agrupados.

    """
    df_grouped = df.groupby(group_column)[agg_column].apply(list).reset_index()
    return df_grouped

In [63]:
def get_frecuency_tags(df, column, n=100):
    """
    Extrae las N etiquetas más frecuentes de un DataFrame donde cada fila contiene una lista de etiquetas.

    Parámetros
    ----------
    df: pandas.DataFrame
        DataFrame que contiene la columna de etiquetas.
    column: str
        Nombre de la columna cuyas celdas son listas de etiquetas.
    n: int, opcional
        Número de etiquetas más frecuentes a devolver (por defecto es 100).

    Retorna
    -------
    list of str
        Lista de las N etiquetas más frecuentes, ordenadas de mayor a menor frecuencia.
    """
    flat_list = [item for sublist in df[column].values for item in sublist]
    keywords = nltk.FreqDist(flat_list)
    frequencies_words = keywords.most_common(n)
    tags_features = [word[0] for word in frequencies_words]
    return tags_features

In [64]:
def most_common(tags):
    """
    Filtra una lista de etiquetas conservando solo aquellas presentes en la lista global `tags_features`.

    Parámetros
    ----------
    tags: list of str
        Lista de etiquetas a filtrar.

    Retorna
    -------
    list of str
        Subconjunto de `tags` que también se encuentran en `tags_features`, conservando el orden original.
    """
    tags_filtered = []
    for i in range(len(tags)):
        if tags[i] in tags_features:
            tags_filtered.append(tags[i])
    return tags_filtered

In [65]:
def clean_text(text):
    """
    Limpia y normaliza una cadena de texto en inglés.

    Convierte el texto a minúsculas, expande contracciones comunes
    (p. ej., "what's" → "what is", "can't" → "can not"), elimina secuencias
    de escape específicas y quita espacios al principio y al final.

    Parámetros
    ----------
    text: str
        Cadena de texto que se desea limpiar y normalizar.

    Retorna
    -------
    str
        Texto procesado: en minúsculas, contracciones expandidas y sin espacios
        iniciales o finales.
    """
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub(r"\'\n", " ", text)
    text = re.sub(r"\'\xa0", " ", text)
    text = text.strip(' ')
    return text

In [66]:
def data_transformation(data):
    """
    Aplica transformaciones de limpieza y filtrado a un DataFrame de preguntas.

    Descripción
    -----------
    - Filtra las etiquetas de cada fila usando `most_common`; si no quedan etiquetas, asigna None.
    - Extrae y limpia el texto HTML de la columna 'Body' usando BeautifulSoup y `clean_text`,
      además de eliminar cualquier etiqueta HTML restante.
    - Asegura que la columna 'Title' sea de tipo cadena.

    Parámetros
    ----------
    data: pandas.DataFrame
        DataFrame que debe contener las columnas:
        - 'Tag': listas de etiquetas por fila.
        - 'Body': texto con posible HTML.
        - 'Title': valores a convertir a texto.

    Retorna
    -------
    pandas.DataFrame
        Mismo DataFrame con las transformaciones aplicadas:
        - 'Tag': lista de etiquetas comunes o None.
        - 'Body': texto limpio sin HTML, en minúsculas y con contracciones expandidas.
        - 'Title': valores convertidos a str.
    """
    data.loc[:, 'Tag'] = data['Tag'].apply(lambda x: most_common(x))
    data.loc[:, 'Tag'] = data['Tag'].apply(lambda x: x if len(x) > 0 else None)
    data = data.dropna(subset=['Tag'])

    data.loc[:, 'Body'] = data['Body'].apply(lambda x: BeautifulSoup(x).get_text())
    data.loc[:, 'Body'] = data['Body'].apply(lambda x: clean_text(x))
    data.loc[:, 'Body'] = data['Body'].apply(lambda x: re.sub('<[^<]+?>','',x))

    data.loc[:, 'Title'] = data['Title'].astype(str)

    return data

### Lectura de datos

In [67]:
questions = read_data('../data/input/Questions.csv')

In [68]:
tags = read_data('../data/input/Tags.csv')
tags = tags.dropna(subset=['Tag'])

### Limpieza de datos y merge de los data sets

In [69]:
questions = drop_columns(questions, ['OwnerUserId', 'CreationDate', 'ClosedDate','Score'])

In [ ]:
grouped_tags = group_tags(tags, "Id", "Tag")

In [ ]:
data_merged = merge_datasets(questions,grouped_tags,'Id')

In [ ]:
tags_features = get_frecuency_tags(data_merged, 'Tag')

### Transformacion de datos

In [73]:
data_merged = data_transformation(data_merged)

### Guardar datos

In [75]:
data_merged.to_parquet('../data/output/StackOverflow.parquet',index=False)